# Description

This notebook is used to request a cropped raster for an area of a WaPOR data layer using WaPOR API.

You will need WaPOR API Token to use this notebook (https://github.com/trngbich/examples_WaPOR_API/blob/master/README.md)

# Step 1: Read APIToken

Get your APItoken from https://wapor.apps.fao.org/profile

In [1]:
import requests
import pandas as pd

path_query=r'https://io.apps.fao.org/gismgr/api/v1/query/'
path_sign_in=r'https://io.apps.fao.org/gismgr/api/v1/iam/sign-in/'

APIToken=input('Your API token: ')

Your API token: Enter your API Token when running this cell


# Step 2: Get Authorization AccessToken

Using the input API token to get AccessToken for authorization

In [2]:
resp_signin=requests.post(path_sign_in,headers={'X-GISMGR-API-KEY':APIToken})
resp_signin = resp_signin.json()
AccessToken=resp_signin['response']['accessToken']

# Step 3: Post Query Load

For more examples of areatimeseries query load 
visit https://io.apps.fao.org/gismgr/api/v1/swagger-ui/examples/AreaStatsTimeSeries.txt

## Define area by coordinate extent

In [3]:
bbox= [37.95883206252312, 7.89534, 43.32093, 12.3873979377346] #latlon
xmin,ymin,xmax,ymax=bbox[0],bbox[1],bbox[2],bbox[3]
Polygon=[
          [xmin,ymin],
          [xmin,ymax],
          [xmax,ymax],
          [xmax,ymin],
          [xmin,ymin]
        ]

query={
  "type": "CropRaster",
  "params": {
    "properties": {
      "outputFileName": "L2_PHE_17s1_s_clipped.tif",
      "cutline": True,
      "tiled": True,
      "compressed": True,
      "overviews": True
    },
    "cube": {
      "code": "L2_PHE_S",
      "workspaceCode": "WAPOR",
      "language": "en"
    },
    "dimensions": [
      {
        "code": "SEASON",
        "values": [
          "S1"
        ]
      },
      {
        "code": "STAGE",
        "values": [
          "SOS"
        ]
      },
      {
        "code": "YEAR",
        "values": [
          "[2017-01-01,2018-01-01)"
        ]
      }
    ],
    "measures": [
      "PHE"
    ],
    "shape": {
      "type": "Polygon",
      "coordinates": [Polygon]
    }
  }
}

## Or define area by passing GeoJSON 

In [ ]:
import ogr
shp_fh=r"D:\Folder\shapefile.shp"

shpfile=ogr.Open(shp_fh)
layer=shpfile.GetLayer()
epsg_code=layer.GetSpatialRef().GetAuthorityCode(None)
shape=layer.GetFeature(0).ExportToJson(as_object=True)['geometry']
shape["properties"]={"name": "EPSG:{0}".format(epsg_code)}#latlon projection            

query={
  "type": "CropRaster",
  "params": {
    "properties": {
      "outputFileName": "L2_PHE_17s1_s_clipped.tif",
      "cutline": True,
      "tiled": True,
      "compressed": True,
      "overviews": True
    },
    "cube": {
      "code": "L2_PHE_S",
      "workspaceCode": "WAPOR",
      "language": "en"
    },
    "dimensions": [
      {
        "code": "SEASON",
        "values": [
          "S1"
        ]
      },
      {
        "code": "STAGE",
        "values": [
          "SOS"
        ]
      },
      {
        "code": "YEAR",
        "values": [
          "[2017-01-01,2018-01-01)"
        ]
      }
    ],
    "measures": [
      "PHE"
    ],
    "shape": shape
  }
}


Post the QueryPayload with AccessToken in Header. In responses, get an url to query job. 

In [4]:
resp_query=requests.post(path_query,headers={'Authorization':'Bearer {0}'.format(AccessToken)},
                                          json=query)
resp_query = resp_query.json()
job_url=resp_query['response']['links'][0]['href']

job_url

'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR/jobs/984f1e50-95f0-42be-af57-f229adaddacf'

# Step 4: Get Job Results.

It will take some time for the job to be finished. When the job is finished, its status will be changed from 'RUNNING' to 'COMPLETED' or 'COMPLETED WITH ERRORS'. If it is COMPLETED, the area time series results can be achieved from Response 'output'.

In [5]:
i=0
print('RUNNING',end=" ")
while i==0:        
    resp = requests.get(job_url)
    resp=resp.json()
    
    if resp['response']['status']=='RUNNING':
        print('.',end =" ")
    if resp['response']['status']=='COMPLETED':
        results=resp['response']['output']
        print(resp['response']['output'])
        i=1
    if resp['response']['status']=='COMPLETED WITH ERRORS':
        print(resp['response']['log'])
        i=1       

RUNNING . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . {'downloadUrl': 'https://io.apps.fao.org/gismgr/download/3c68d62d-b720-4de4-9f63-1fbb3fa637a0/L2_PHE_17s1_s_clipped.tif'}
